# Simple wave sediment modelling

:::{note}
Using a wave propagation model based on linear wave theory, we investigate the cumulative effect of a given wave forcing condition on erosional and depositional trend across a regional coastal system.</div>
:::

***

**Content:**
- [Model set-up](#setup)
- [Model initialisation function](#init)
- [Waves evolution](#evol)
- [Sediment entrainment, transport & deposition](#sed)

***

Loading required modules for this exercise. The main module here is **wavesed** a piece of code that was developed for this course and that can be found on [GitHub](https://github.com/pyReef-model/wavesed)

In [5]:
!conda install -c conda-forge compilers --y

Solving environment: done

## Package Plan ##

  environment location: /Users/getafix/anaconda3/envs/coast

  added / updated specs:
    - compilers


The following NEW packages will be INSTALLED:

  c-compiler         conda-forge/osx-64::c-compiler-1.1.3-h35c211d_0
  clangxx_osx-64     conda-forge/osx-64::clangxx_osx-64-11.1.0-h7e1b574_0
  compilers          conda-forge/osx-64::compilers-1.1.3-h694c41f_0
  cxx-compiler       conda-forge/osx-64::cxx-compiler-1.1.3-h926bf3e_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [6]:
!f2py

Usage:

1) To construct extension module sources:

      f2py [<options>] <fortran files> [[[only:]||[skip:]] \
                                        <fortran functions> ] \
                                       [: <fortran files> ...]

2) To compile fortran files and build extension modules:

      f2py -c [<options>, <build_flib options>, <extra options>] <fortran files>

3) To generate signature files:

      f2py -h <filename.pyf> ...< same options as in (1) >

Description: This program generates a Python C/API file (<modulename>module.c)
             that contains wrappers for given fortran functions so that they
             can be called from Python. With the -c option the corresponding
             extension modules are built.

Options:

  --2d-numpy       Use numpy.f2py tool with NumPy support. [DEFAULT]
  --2d-numeric     Use f2py2e tool with Numeric support.
  --2d-numarray    Use f2py2e tool with Numarray support.
  --g3-numpy       Use 3rd generation f2py from the separ

In [2]:
#!git clone --depth=1 https://github.com/matplotlib/legacycontour.git; cd legacycontour; python3 setup.py install; cd ..; rm -rf legacycontour

In [7]:
!cd wavesed/; f2py -c -m ocean ocean.f90    

running build
running config_cc
unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
build_src
building extension "ocean" sources
f2py options: []
f2py:> /var/folders/px/dgn5m62n7bjbrc1d3cplcv7m0000gn/T/tmprpbu8pqo/src.macosx-10.9-x86_64-3.8/oceanmodule.c
creating /var/folders/px/dgn5m62n7bjbrc1d3cplcv7m0000gn/T/tmprpbu8pqo/src.macosx-10.9-x86_64-3.8
Reading fortran codes...
	Reading file 'ocean.f90' (format:free)
Post-processing...
	Block: ocean
			Block: ocean
				Block: airymodel
				Block: transport
				Block: diffusion
Post-processing (stage 2)...
	Block: ocean
		Block: unknown_interface
			Block: ocean
				Block: airymodel
				Block: transport
				Block: diffusion
Building modules...
	Building module "ocean"...
		Constructing F90 module support for "ocean"...
		  Variables: pi onpi pi2 pion2 onpi2 grav
			Constructing wrapper func

In [8]:
import time
import numpy as np
import wavesed.wave as ocean

%matplotlib inline

ImportError: cannot import name 'dedent' from 'matplotlib.cbook' (/Users/getafix/anaconda3/envs/coast/lib/python3.8/site-packages/matplotlib/cbook/__init__.py)

## <a name="setup"></a>  Model set-up


The model requires several parameters to be defined before being run. The most important one being the initial bathymetry. For this example, we will use the region around [Fraser Island](https://www.google.com.au/maps/@-24.8590806,152.2555941,236264m/data=!3m1!1e3?hl=en) at a resolution of 1000 m. The _digital elevation model_ (DEM) comes from the [Project 3DGBR](http://eatlas.org.au/data/uuid/200aba6b-6fb6-443e-b84b-86b0bbdb53ac).

<img src="../../figs/ex4-fig1.png" width="60%">

The file is called **FraserDEM.csv** and is located in the **data** folder. It is made of 3 columns containing the X, Y, and Z coordinates. 

### Model domain / grid parameters

It is possible to change the initial resolution by using the `rfac` parameter, as an example:

+ `rfac = 1` corresponds to a model at 1000 m resolution (similar to the initial DEM)
+ `rfac = 2` transforms the dataset to 2000 m resolution 

In [ ]:
# Bathymetric filename
bfile = '../../dataset/FraserDEM.csv'
# Resolution factor
rfac = 1 

### Definition of wave parameters

Now we need to define the forcing parameters for our simulation:

+ `H0` the deep water wave height in [m],
+ `dir` the deep water wave source direction at the boundary in degrees (angle is defined counterclock wise from horizontal axis)
+ `wbase` the maximum depth for wave influence on seabed [m]
+ `slvl` the sea-level position in [m]

In [ ]:
# Wave height
H0 = 2
# Wave direction at boundary 
dir = 0
# Maximum depth for wave influence (m)
wbase = 10
# Sea level position (m)
slvl = 0.
# Output figure name
outname = 'wdir0'

### Definition of sediment parameters

To estimate wave-induced sediment transport, this simple model only accounts for a single grain size that needs to be defined (`d50`). 

There are also additional parameters that can be set to perform longshore and diffusive transport but we will use the default values for this example...

In [ ]:
# Mean grain size diameter in m
d50 = 0.0001

## <a name="init"></a>  Model initialisation function

Now that all the parameters are defined in our python environment, we can initialise the simulation by calling the wavesed module:

In [ ]:
#help(ocean.wave.__init__)

In [ ]:
wavesed = ocean.wave(filename=bfile, wavebase=wbase, resfac=rfac, dia=d50)

As you can see from the command above there are still some parameters that have not been passed to the module:

+ `slvl`
+ `dir`
+ `H0`

These parameters are sent separately so that multiple climatic conditions could be simulate more easily.

First we define the sea-level position `slvl`.

In [ ]:
t0 = time.clock()
wavesed.findland(slvl)
print('Definition of shoreline position took (s):',time.clock()-t0)

Initialising wave boundary direction `dir` is then done with the `wavesource` function:

In [ ]:
wdir = wavesed.wavesource(dir)

## <a name="evol"></a>  Waves evolution

The waves are then transformed using the **linear wave theory** from deep (set-up conditions) to shallow water assuming shore-parallel depth contours. 

The orientation of wave fronts is determined by wave _celerity_ and _refraction_ due to depth variations. 

Travel time in the domain is calculated from **Huygen's principle** (using an order $\sqrt{5}$ approximation).

Assuming no refraction or loss of energy due to bottom friction, wave power $P$ is conserved from deep to shallow water. 

In [ ]:
#help(wavesed.cmptwaves)

In [ ]:
t0 = time.clock()
wavesed.cmptwaves(src=wdir, h0=H0, sigma=1.)
print('Wave evolution computation took (s): ',time.clock()-t0)

### Plotting simulated wave characteristics

A series of graph can be visualised to show the following characteristics:

+ _bathy_: coarse bathymetric map
+ _wlength_: coarse wave lenght map
+ _travel_: coarse wave travel time map
+ _wcelerity_: coarse wave celerity map
+ _wheight_: coarse wave height map
+ _wpower_: coarse wave power map
+ _ubot_: coarse wave induced bottom velocity map
+ _shear_: coarse wave induced shear stress map

In [ ]:
#help(wavesed.plotData)

In [ ]:
size = (10,10)
# i1 = 0 
# i2 = -1
# j1 = 0
# j2 = -1

# Zooming to a specific region
i1 = 80 
i2 = 250 
j1 = 0
j2 = 170 

wavesed.plotData(data='bathy', figsize=size, vmin=-100, vmax=100, 
                 fontsize=10, imin=i1, imax=i2, jmin=j1, jmax=j2)

wavesed.plotData(data='travel', figsize=size, tstep=100, vmin=0, vmax=5.e4, 
                 fontsize=10, imin=i1, imax=i2, jmin=j1, jmax=j2)

wavesed.plotData(data='wcelerity', figsize=size, vmin=0, vmax=15, 
                 fontsize=10, stream=3, imin=i1, imax=i2, jmin=j1, jmax=j2)

wavesed.plotData(data='ubot', figsize=size, vmin=0, vmax=2, 
                 fontsize=10, imin=i1, imax=i2, jmin=j1, jmax=j2)

wavesed.plotData(data='shear', figsize=size, vmin=-10, vmax=10, 
                  fontsize=10, imin=i1, imax=i2, jmin=j1, jmax=j2)

## <a name="sed"></a>  Sediment entrainment, transport & deposition

**Sediment entrainment** relates to wave induced shear stress. 

The transport is computed according to both wave direction and longshore transport. Deposition is dependent of shear stress and diffusion.

In [ ]:
#help(wavesed.cmptsed)

In [ ]:
t0 = time.clock()
wavesed.cmptsed(sigma=1.)
print('Sediment erosion/deposition computation took (s): ',time.clock()-t0)

### Plotting morphological changes

Using the same plotting function (`plotData`), we can now plot the following metrics:

+ _ent_: coarse sediment entrainment map
+ _dz_: coarse erosion/deposition map

In [ ]:
#help(wavesed.plotData)

In [ ]:
size = (10,10)
# i1 = 0 
# i2 = -1
# j1 = 0
# j2 = -1

# Zooming to a specific region
i1 = 80 
i2 = 250 
j1 = 0
j2 = 170

wavesed.plotData(data='bathy', figsize=size, vmin=-100, vmax=100, 
                 fontsize=10, stream=0, imin=i1, imax=i2, jmin=j1, jmax=j2)

wavesed.plotData(data='ent', figsize=size, vmin=0., vmax=5., 
                 fontsize=10, stream=0, imin=i1, imax=i2, jmin=j1, jmax=j2)

wavesed.plotData(data='dz', figsize=size, vmin=-1., vmax=1., 
                 fontsize=10, stream=0, imin=i1, imax=i2, jmin=j1, jmax=j2) #, save=outname)


## <a name="save"></a>  Saving wave/sedimentation data



In [ ]:
#waveparams.outputCSV(filename='erodep.csv', seddata=1)

***

<div class="alert alert-block alert-info">
1- Change the initial wave direction conditions to 90 and then to 270. Evaluate the impact on the morphological patterns. 
</div>

<div class="alert alert-block alert-info">
2- Change the resolution factor to see how it impacts on erosion/deposition resolution.</div>

***